# Find similar objects in XMM for objects in simbad, GEV and TEV catalogs
First, we import all the packages that we will need.

In [3]:
import warnings
import pandas as pd
import numpy as np
from getsimbad import get_simbad_data
from gevtev import compare_gev_tev_data
from astropy.io import fits
from definitions import *

warnings.filterwarnings("ignore")

This constant determines a threshold for the similarity between the objects:
$$|glat_{1} - glat_{2}| <= \epsilon$$
$$\epsilon = 0,001$$

In [4]:
# _epsilon = 0.01
# _epsilon = 0.05
_epsilon = 0.12

In [5]:
_epsilon

0.12

In [6]:
_path_xmm = 'data/3XMM_DR7cat_slim_v1.0.fits.gz'
_names_xmm = [
    "SC_RA",
    "SC_DEC",
    "SC_POSERR",
    "SC_EP_1_FLUX",
    "SC_EP_2_FLUX",
    "SC_EP_3_FLUX",
    "SC_EP_4_FLUX",
    "SC_EP_5_FLUX",
    "SC_EP_1_FLUX_ERR",
    "SC_EP_2_FLUX_ERR",
    "SC_EP_3_FLUX_ERR",
    "SC_EP_4_FLUX_ERR",
    "SC_EP_5_FLUX_ERR",
    ] 

In [7]:
hdul_xmm = fits.open(_path_xmm)
cat_xmm = hdul_xmm[1].data

In [8]:
def create_pandas_frames(cat, cat_name):   
    """
    Creates pandas dataframes with the same values as in cat, adding to columns cat_name.
    
    Arguments:
    cat -- rec array with catalog data
    cat_name -- short catalog name
    
    Returns:
    data -- pandas DataFrame with catalog data
    """
    
    data = pd.DataFrame.from_records(cat.tolist(), columns=cat.dtype.names)
    match_names = {}
    for i in data.columns:
        match_names.update({i : cat_name + i})
    data = data.rename(columns = match_names)
    
    return data

In [9]:
data_xmm = create_pandas_frames(cat_xmm, "xmm_")
data_xmm.head()

,xmm_SRCID,xmm_IAUNAME,xmm_SC_RA,xmm_SC_DEC,xmm_SC_POSERR,xmm_SC_DET_ML,xmm_SC_EP_1_FLUX,xmm_SC_EP_1_FLUX_ERR,xmm_SC_EP_2_FLUX,xmm_SC_EP_2_FLUX_ERR,...,xmm_SC_SUM_FLAG,xmm_SC_EP_8_FMIN,xmm_SC_EP_8_FMIN_ERR,xmm_SC_EP_8_FMAX,xmm_SC_EP_8_FMAX_ERR,xmm_MJD_FIRST,xmm_MJD_LAST,xmm_N_DETECTIONS,xmm_CONFUSED,xmm_WEBPAGE_URL
0,206931901010113,b'3XMM J000000.0-552108',0.000395,-55.352442,2.31261,11.961400,5.975190e-17,1.080930e-16,7.739470e-16,2.520810e-16,...,0,6.018350e-15,5.191330e-15,6.018350e-15,5.191330e-15,56086.742951,56087.588194,1,70,b'http://xmm-catalog.irap.omp.eu/source/206931...
1,204033901010030,b'3XMM J000000.2+623122',0.001138,62.523022,1.81998,24.577000,2.084380e-17,6.754120e-17,1.423010e-17,9.893530e-17,...,0,7.443400e-15,1.952370e-15,7.443400e-15,1.952370e-15,54134.485278,54135.043600,1,70,b'http://xmm-catalog.irap.omp.eu/source/204033...
2,201253101010082,b'3XMM J000000.3-250629',0.001307,-25.108284,2.15064,10.571600,1.946440e-16,2.242400e-16,1.116210e-15,3.949710e-16,...,0,5.887980e-15,3.771330e-15,5.887980e-15,3.771330e-15,51696.440880,51696.788090,1,70,b'http://xmm-catalog.irap.omp.eu/source/201253...
3,206931901010049,b'3XMM J000000.3-552139',0.001344,-55.361047,1.86001,48.571301,7.582340e-16,2.276970e-16,1.875440e-15,3.475400e-16,...,0,1.669370e-14,6.452240e-15,1.669370e-14,6.452240e-15,56086.742951,56087.588194,1,70,b'http://xmm-catalog.irap.omp.eu/source/206931...
4,203025805010073,b'3XMM J000000.7-321352',0.003264,-32.231256,1.94055,14.014100,2.411770e-16,1.397010e-16,7.000080e-16,2.408750e-16,...,0,6.316080e-15,3.759480e-15,6.316080e-15,3.759480e-15,53534.288912,53534.827627,1,70,b'http://xmm-catalog.irap.omp.eu/source/203025...


In [10]:
del cat_xmm

Load data from gevtevsimbad, gevsimbad and tevsimbad:

In [11]:
common_data = pd.read_csv("data/gevtev_simbadclasses.txt", sep='\t', encoding='utf-8')

In [12]:
common_data[['tev_pos_ra', 'tev_pos_dec', 'gev_RAJ2000', 'gev_DEJ2000', 'tev_glat', 'gev_GLAT', 'tev_glon', 'gev_GLON']]

,tev_pos_ra,tev_pos_dec,gev_RAJ2000,gev_DEJ2000,tev_glat,gev_GLAT,tev_glon,gev_GLON
0,1.650035,72.783002,1.765600,73.049896,10.203682,10.460557,119.580254,119.662102
1,3.483564,-18.901846,3.476100,-18.896601,-78.086937,-78.078400,74.632011,74.626923
2,6.339723,64.140802,6.432600,64.069298,1.412930,1.337755,120.092361,120.125992
3,8.393293,-19.359207,8.402200,-19.363800,-81.216103,-81.224083,94.174644,94.212082
4,8.968984,59.834499,8.989100,59.831799,-2.978120,-2.981383,120.975426,120.985832
5,11.888062,-25.288798,11.886600,-25.275000,-87.964546,-87.951553,97.363853,97.501717
6,24.135832,39.099778,24.130501,39.096298,-22.939846,-22.943954,132.416458,132.413315
7,28.165054,1.788161,28.162001,1.807500,-57.540936,-57.524715,152.377487,152.357361
8,31.404325,64.828299,31.395700,64.815002,3.083848,3.070137,130.717819,130.718506
9,35.272806,35.937101,35.280102,35.940701,-23.486858,-23.481199,142.601715,142.606796


In [13]:
def create_matrix_common(data1, data2, epsilon, data1_ra, data1_dec, data2_ra, data2_dec):
    """This function looks for the same objects in GeV and TeV and Simbad catalogs 
    
    Return: a matrix with association: m[i][j] = True if object i from cat1
            is close enough to object j from cat2
    
    cat1, cat2 - a pandas dataFrames 
    epsilon(double) - distance accepted as equivalence
    """

    ra_dif_matrix = np.dot(np.vstack((data1[data1_ra].T, -np.ones_like(data1[data1_ra].T))).T, 
                           np.vstack((np.ones_like(data2[data2_ra].T), data2[data2_ra].T)))
    dec1 = data1[data1_dec].reshape(-1, 1)
    dec2 = data2[data2_dec].reshape(-1, 1)

    SEP = np.cos((ra_dif_matrix) * np.pi / 180) * np.cos(dec1 * np.pi / 180) * np.cos(dec2.T * np.pi / 180)
    SEP += np.sin(dec1 * np.pi / 180) * np.sin(dec2.T * np.pi / 180) # returns values between 0 and pi radians
    SEP = np.arccos(SEP) * 180. / np.pi
     
    pairs_matrix = (SEP < epsilon)
    return pairs_matrix

In [14]:
from tqdm import tqdm

In [15]:
def create_common_data(data1, data2, pairs_matrix, cat1_col1, cat1_col2, cat2_col1, cat2_col2):
    """The function adds objects found in both catalogs.
    """
    vector_association = np.where(np.sum(pairs_matrix, axis=0) > 0)[0]
    pd_common = pd.DataFrame()
    for i in tqdm(vector_association):
        data1_join = (pairs_matrix[:, i] > 0)*(i + 1) - 1
        data1["join"] = data1_join
        pd_common0 = pd.merge(data1, data2, right_index=True, left_on='join', how='inner')
        if (len(pd_common)):
            pd_common = pd_common.append(pd_common0)
            del pd_common["join"]
            del data1["join"]
        else:
            pd_common = pd_common0.copy()
    
      
    array_non_duplicate = [cat1_col1, cat1_col2, cat2_col1, cat2_col2]
    pd_common = pd_common.drop_duplicates(array_non_duplicate)
    pd_common = pd_common.reset_index()

    return pd_common

In [16]:
ra_gc = 266.416817
dec_gc =  -29.007825

In [17]:
def angsep(ra1, dec1, ra2, dec2):
    SEP = np.cos(dec1 * np.pi / 180) * np.cos(dec2 * np.pi / 180) * np.cos((ra1 - ra2) * np.pi / 180)
    SEP += np.sin(dec1 * np.pi / 180) * np.sin(dec2 * np.pi / 180) #returns values between 0 and pi radians
    SEP = np.arccos(SEP)
    return SEP * 180. / np.pi

In [18]:
print(common_data.shape)
common_data = common_data[angsep(common_data['tev_pos_ra'], common_data['tev_pos_dec'], ra_gc, dec_gc) > 1]
print(common_data.shape)

(199, 170)
(183, 170)


In [19]:
# common_data[np.logical_and(np.abs(common_data['tev_pos_ra'] - gal_ra)> 1, np.abs(common_data['tev_pos_ra']-gal_dec) > 1)]

In [20]:
_epsilon

0.12

In [21]:
matrix = create_matrix_common(data_xmm, common_data, _epsilon, 'xmm_SC_RA', 'xmm_SC_DEC', "tev_pos_ra", "tev_pos_dec")
#pd_common_gevtevsimbad = create_common_data(common_data, pd_simbad, matrix, 'tev_glat', 'tev_glon', "s_glat", "s_glon")
#print("There are " + str(len(pd_common_gevtevsimbad)) + " common objects in the three catalogs.")

In [22]:
pd_common_gevtevxmm = create_common_data(data_xmm, common_data, matrix, 'xmm_SC_RA', 'xmm_SC_DEC', "tev_pos_ra", "tev_pos_dec")

100%|██████████| 138/138 [00:22<00:00,  6.08it/s]


In [23]:
print("There are " + str(len(pd_common_gevtevxmm)) + " common objects in the three catalogs.")

There are 4975 common objects in the three catalogs.


In [24]:
pd_common_gevtevxmm[['xmm_SC_RA', 'xmm_SC_DEC', "tev_pos_ra", "tev_pos_dec"]]

,xmm_SC_RA,xmm_SC_DEC,tev_pos_ra,tev_pos_dec
0,1.592070,72.833347,1.650035,72.783002
1,1.611118,72.897266,1.650035,72.783002
2,1.693981,72.848707,1.650035,72.783002
3,1.699443,72.885301,1.650035,72.783002
4,1.720686,72.846647,1.650035,72.783002
5,1.754500,72.837654,1.650035,72.783002
6,6.187147,64.119458,6.339723,64.140802
7,6.190435,64.127330,6.339723,64.140802
8,6.193961,64.130024,6.339723,64.140802
9,6.195151,64.124451,6.339723,64.140802


In [25]:
pd_common_gevtevxmm.to_csv("data/gevtevsimbadxmm_012.txt", sep='\t')